In [7]:
import pandas as pd
import pickle
import os
import numpy as np
import random
from colog.colog import colog
from datetime import datetime, timedelta
from tqdm import tqdm
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
c = colog()

In [2]:
def load_data_one_stock(path):
  load_data = pd.DataFrame()
  try:
    file = open(path, 'rb')
    load_data= pickle.load(file)
    file.close()
  except Exception as e:
    print(e)
  return load_data

In [8]:
# SETTINGS
# stock_name_list  = ['AAPL','V', 'AMD', 'MSFT', 'MA', 'NVDA']
# stock_name_list  = ['GOOGL','CVS', 'AMZN', 'CRM', 'DIS', 'MCD']

# stock_name_list  = ['AAPL','V', 'AMD', 'MSFT', 'MA', 'NVDA'] + ['GOOGL','CVS', 'AMZN', 'CRM', 'DIS', 'MCD']
stock_name_list = []
stock_name_list += ['GOOG','JPM','XOM','UNH','JNJ','V','AVGO','PG','LLY','MA','HD','CVX','MRK', 
                       'PEP','COST','ABBV','ADBE','KO','CRM','WMT','MCD','CSCO','BAC','PFE','TMO','ACN','NFLX','ABT','AMD','LIN','ORCL','CMCSA',
                       'TXN','DIS','WFC','DHR','PM','NEE','VZ','INTC','RTX','HON','LOW','UPS','INTU','SPGI','NKE','COP','QCOM','BMY','CAT','UNP','BA','ISRG',
                        'GE','IBM','AMGN','AMAT','MDT','SBUX','PLD','NOW','MS','DE','BLK','GS','T','LMT','AXP','SYK','ADI','TJX','ELV','MDLZ','GILD','ADP','MMC',
                        'C','AMT','CVS','VRTX','SCHW','LRCX','MO','TMUS','SLB', 'ETN', 'ZTS', 'CI', 'PYPL']

stock_name_list += ['FI','CB','SO','REGN','BSX','EQIX','BDX','PANW','DUK','EOG','MU','AON','ITW','CSX','SNPS','PGR','APD','KLAC','CME','NOC','CDNS','ICE',
                       'CL','SHW','WM','HCA','TGT','FCX','FDX','F','MMM','CMG','EW','GM','MCK','NXPI','MCO','NSC','HUM','EMR','DXCM','PNC','PH','MPC','APH',
                       'ROP','FTNT','MCHP','PXD','USB','CCI','MAR','MSI','GD','PSA','JCI','PSX','SRE','ADSK','AZO','TDG','ECL','AJG','KMB','TEL','TT','AEP','EL','PCAR',
                       'OXY','TFC','CARR','D','IDXX','GIS','ON','COF','ADM','MNST','NUE','CTAS','AIG','EXC','VLO','MRNA','ANET','WMB','O','STZ','IQV','HLT','CHTR','WELL',
                       'BIIB','SPG','MSCI','DHI','ROK']

stock_name_list_selected = ['EXC', 'MCHP', 'ADI', 'WFC', 'EW', 'MNST', 'FTNT', 'AIG', 'BAC', 'TXN', 
                        'TT', 'ECL', 'GOOG', 'LRCX', 'GS', 'PSX', 'MA', 'CVS', 'PG', 'BA', 'DE',
                         'PGR', 'PFE', 'TFC', 'ADM', 'NOW', 'SRE', 'PLD', 'TEL', 'MCD', 'ADP', 'NOC', 
                         'F', 'REGN', 'KO', 'ZTS', 'WM', 'CVX', 'NFLX', 'CCI', 'MDLZ', 'WMB', 'BSX', 'CARR',
                         'PM', 'JNJ', 'MRK', 'COP', 'AON', 'COF', 'SYK', 'IQV', 'GD', 'V', 'VRTX', 'CSCO',
                         'USB', 'TGT', 'DUK', 'MDT', 'ABT', 'PSA', 'KMB', 'MCK', 'KLAC', 'C', 'AEP', 'CHTR',
                         'EOG', 'AJG', 'MU', 'ADBE', 'QCOM', 'ADSK', 'PCAR', 'GM', 'SBUX', 'AXP', 'CTAS',
                         'FI', 'SPG', 'ORCL', 'NSC', 'IBM', 'INTU', 'BDX']

folder_saved_models = f'historical_data\\period2yinterval1h'
parent_path = os.getcwd()
buy_money = 2000
commission_value = 0.0005
training_part = 0.4
valid_part = 0.2
test_part = 0.4

with open(f'{folder_saved_models}/df_stats.plk', 'rb') as file:
  df_stats = pickle.load(file)

In [42]:
ticker = 'PGR'
file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
path = os.path.join(parent_path, folder_saved_models, file_name)
df = load_data_one_stock(path)

# df = px.data.gapminder().query("country=='Canada'")
# fig = px.line(x = df_stats[ticker].index, y="amps_avg100", title='')
# fig.show()
df['max100'] = df['close'].rolling(window=100).max()
df['max100'] = df['max100'].bfill()
df['min100'] = df['close'].rolling(window=100).min()
df['min100'] = df['min100'].bfill()
df['current_amp100'] = df['max100'] / df['min100']
df['norm100'] = (df['current_amp100']- 1) / (df_stats[ticker]['amps_avg100'].iloc[-1] - 1)
df_stats[ticker]['speed100_avg'] = df_stats[ticker]['speed100'].rolling(window=30).mean()
df_stats[ticker]['speed30_avg'] = df_stats[ticker]['speed30'].rolling(window=30).mean()

# df_stats['norm100'] =   norm100 = (current_amp100 - 1) / (df_stats[ticker]['amps_avg100'].iloc[-1] - 1)

subfig = make_subplots(specs=[[{"secondary_y": True}]])
fig1 = px.line(df, x=df.index, y='close')
# fig2 = px.line(df, x=df.index, y='norm100')
fig2 = px.line(df_stats[ticker], x=df_stats[ticker].index, y='speed30_avg')
fig3 = px.line(df_stats[ticker], x=df_stats[ticker].index, y='speed100_avg')
# fig4 = px.line(df_stats[ticker], x=df_stats[ticker].index, y='speed30')
# fig4 = px.line(df_stats[ticker], x=df_stats[ticker].index, y=[]'speed30_avg')
fig2.update_traces(yaxis="y2", line_color='red')
fig3.update_traces(yaxis="y2", line_color='orange')


subfig.add_traces(fig1.data + fig2.data + fig3.data)

In [45]:
def is_near_global_max(i, k=400, prt=70):

  if i > k:
   gmax = max(df['close'].iloc[i - k: i])
   reference_point = df['close'].iloc[i - k]
   gmin = min(df['close'].iloc[i - k: i])
  else:
   reference_point = df['close'].iloc[0]
   gmax = max(df['close'].iloc[0: i])
   gmin = min(df['close'].iloc[0: i])
  # if gmax == reference_point:
  #   result = False
  # elif 100 * (df['close'].iloc[i] - reference_point) / (gmax - reference_point) > prt:
  #   result = True
  # else:
  #   result = False
  result = 100 * (df['close'].iloc[i] - gmin) / (gmax - gmin) > prt

  return result

def global_max_prop(i, k=40):

  if i > k:
   gmax = max(df['close'].iloc[i - k: i])
   gmin = min(df['close'].iloc[i - k: i])
  else:
   gmax = max(df['close'].iloc[0: i])
   gmin = min(df['close'].iloc[0: i])

  return gmax / gmin

def number_red_candles(df, i, k=11):

  if i < k:
    number_red_candles = (df['ha_colour'][0 : i] == 'red').sum()
  else:
    number_red_candles = (df['ha_colour'][i - k : i] == 'red').sum()
  return number_red_candles

def number_green_candles(df, i, k=11):


  if i < k:
    number_green_candles = (df['ha_colour'][0 : i] == 'green').sum()
  else:
    number_green_candles = (df['ha_colour'][i - k : i] == 'green').sum()
  return number_green_candles

def more_than_pct(value1, value2, pct):
  if 100 * (value1 / value2 - 1) > pct:
    return True
  else:
    return False

def maximum(df, i, k=20):
  if i < k:
    return np.amax([df['close'].iloc[0 : k].max(), df['open'].iloc[0 :k].max(), df['close'].iloc[i], df['open'].iloc[i]])
  else: 
    return np.amax([df['close'].iloc[i -k : i].max(), df['open'].iloc[i - k : i].max(), df['close'].iloc[i], df['open'].iloc[i]])

def miminum(df, i, k=20):  
  if i < k:
    return np.amin([df['close'].iloc[0 : k].min(), df['open'].iloc[0 : k].min(), df['close'].iloc[i], df['open'].iloc[i]])
  else:
    return np.amin([df['close'].iloc[i - k : i].min(), df['open'].iloc[i - k : i].min(), df['close'].iloc[i], df['open'].iloc[i]])

def norm(df, i, k=20):
  min = miminum(df,i,k)
  max = maximum(df,i,k)
  return (df['close'].iloc[-1] - min) / (max - min + 0.0001)


def get_amps_old(df, i):

  amps7 = []
  amps30 = []
  amps100 = []
  if i > 300:
    for k in range(i - 300, i):
      min = miminum(df, k, 7)
      max = maximum(df, k, 7)
      amps7.append(max / min)
      min = miminum(df, k, 30)
      max = maximum(df, k, 30)
      amps30.append(max / min)
      min = miminum(df, k, 100)
      max = maximum(df, k, 100)
      amps100.append(max / min)
    
    amp7 = np.sum(amps7) / len(amps7)
    amp30 = np.sum(amps30) / len(amps30)
    amp100 = np.sum(amps100) / len(amps100)
  else:
    amp7 = 0
    amp30 = 0
    amp100 = 0

  return amp7, amp30, amp100

def get_amps(df, p=7):
  min = []
  max = []
  amp = []
  amps_avg = []
  
  for i in range(df.shape[0]):
    if i < p:
      min.append(miminum(df, i, p))
      max.append(maximum(df, i, p))
    else:
      min.append(miminum(df, i, p))
      max.append(maximum(df, i, p))
    amp.append(max[i] / min[i])
    if i < 300:
      amps_avg.append(sum(amp[:i+1]) / len(amp[:i+1]))
    else:
      amps_avg.append(sum(amp[i - 300 : i + 1]) / len(amp[i - 300 : i + 1]))
    
  return amps_avg

def get_speed(df, p):
  speed100 = []
  for i in range(df.shape[0]):
    if i < p:
      speed100.append(df['close'].iloc[i] / df['close'].iloc[0])
    else:
      speed100.append(df['close'].iloc[i] / df['close'].iloc[i - p])
  return speed100

def extremum_minimum(ser, i, window=10):

  # if ser.iloc[i] > ser.iloc[i - 1] \
  #   and ser.iloc[i] > ser.iloc[i - 2] \
  #   and ser.iloc[i - 2] < ser.iloc[i - 4] \
  #   and ser.iloc[i - 2] < ser.iloc[i - 5]:
  #   return True
  # else:
  #   return False
  if ser.iloc[i - window: i].sum() > ser.iloc[i - 2 * window : i - window].sum() \
    and ser.iloc[i - 3 * window: i - 2 * window].sum() > ser.iloc[i - 2 * window : i - window].sum():
    return True
  else:
    return False

In [117]:
if False:
  df_stats = {}
  for ticker in tqdm(stock_name_list):
    try:
      file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
      path = os.path.join(parent_path, folder_saved_models, file_name)
      df = load_data_one_stock(path)
      speed7 = get_speed(df, 7)
      speed30 = get_speed(df, 30)
      speed100 = get_speed(df, 100)
      amps_avg7 = get_amps(df, 7)
      amps_avg30 = get_amps(df, 30)
      amps_avg100 = get_amps(df, 100)
      df['max100'] = df['close'].rolling(window=100).max()
      df['max100'] = df['max100'].bfill()
      df['min100'] = df['close'].rolling(window=100).min()
      df['min100'] = df['min100'].bfill()
      df['current_amp100'] = df['max100'] / df['min100']
      # df['norm100'] = (df['current_amp100']- 1) / (df_stats[ticker]['amps_avg100'].iloc[-1] - 1)
      data =  {'amps_avg7': amps_avg7, 'amps_avg30': amps_avg30, 'amps_avg100': amps_avg100,
              'speed7': speed7, 'speed30': speed30, 'speed100': speed100}
      df_stats[ticker] = pd.DataFrame(data,index=df.index)
      df_stats[ticker]['speed100_avg30'] = df_stats[ticker]['speed100'].rolling(window=30).mean()
      df_stats[ticker]['speed30_avg30'] = df_stats[ticker]['speed30'].rolling(window=30).mean()
      df_stats[ticker]['speed30_avg30'] = df_stats[ticker]['speed30_avg30'].bfill()
      df_stats[ticker]['speed100_avg30'] = df_stats[ticker]['speed100_avg30'].bfill()
    except Exception as e:
      print(e)

  with open(f'{folder_saved_models}/df_stats.plk', 'wb') as file:
    pickle.dump(df_stats, file) 

 73%|███████▎  | 138/188 [15:00<05:16,  6.34s/it]

[Errno 2] No such file or directory: 'f:\\Git Projects\\HeikenAshiInvestingModel\\historical_data\\period2yinterval1h\\df_stock_PXD_period2y_interval1h.pkl'
'close'


100%|██████████| 188/188 [20:02<00:00,  6.40s/it]


In [7]:
if False:
  amps7, amps30, amps100 = {}, {}, {}
  speeds_100 = {}

  for ticker in tqdm(stock_name_list):
  # for ticker in ['GOOG']:
    file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
    path = os.path.join(parent_path, folder_saved_models, file_name)
    df = load_data_one_stock(path)
    amp7, amp30, amp100 = get_amps(df, df.shape[0])
    # speed = df['close'].iloc[i - 1] / df['close'].iloc[i - 100] 
    amps7[ticker] = amp7
    amps30[ticker] = amp30
    amps100[ticker] = amp100

In [40]:
def one_stock_simulation(df, ticker, mode, _gain_coef = 1.005, _lose_coef = 0.97, print_sim_results=False, print_trans_history=False):
  '''
    Parameters:
      mode: train / full / test

    Returns:
      profits, wins, loses 
  '''

  history_log = {'number success': 0,
                'number fails': 0,
                'number wins': 0,
                'proportion': 0,
                'profit': 0,
                'transactions': [] }  
  
  sell_finished = True
  buy_order_placed = False
  buy_price = 0
  
  if mode == 'train':
    range_ = range(2, int(df.shape[0] * training_part))
  if mode == 'valid':
    range_ = range(int(df.shape[0] * training_part) + 1, int(df.shape[0] * (valid_part + training_part)))
  if mode == 'full': 
    range_ = range(2, df.shape[0] - 1)
  if mode == 'test':
    range_ = range(int(df.shape[0] * (valid_part + training_part)) + 1, df.shape[0] - 1)

  buy_i = 0
  conditions = []
  CM930, CM1030, CM1130, CM1230, CM1330 = 10000, 10000, 10000 ,10000 ,10000
  
  for i in range_:

    profit = df['high'].iloc[i] / (buy_price + 0.0001)

    min7  = miminum(df, i - 1, 7)
    min30  = miminum(df, i - 1 , 30)
    min100  = miminum(df, i - 1, 100)
    max7  = maximum(df, i - 1, 7)
    max30  = maximum(df, i - 1, 30)
    max100  = maximum(df, i - 1, 100)
    max300  = maximum(df, i - 1, 300)
    cv = df['close'].iloc[i - 1]
    dist_from_max_7 = max7 / cv
    dist_from_max_30 = max30 / cv
    dist_from_max_100 = max100 / cv

    if i >= buy_i:

      if sell_finished == False and df['high'].iloc[i] / buy_price > _gain_coef:
        # sell_price = float(df['high'].iloc[i])
        sell_price = buy_price * _gain_coef
        history_log['number success'] += 1
        number_buy_stocks = int(buy_money / buy_price)
        history_log['profit'] += (sell_price - buy_price) * number_buy_stocks - commission_value * (sell_price + buy_price) * number_buy_stocks
        history_log['number wins'] += 1
        history_log['transactions'].append({
          'type': 'success',
          'buy_price' : round(buy_price, 2),
          'sell_price': round(sell_price, 2), 
          'buy_time' : buy_time,
          'sell_time' : df['high'].index[i]
      })
        sell_finished = True
        buy_order_placed = False

      if sell_finished == False and df['low'].iloc[i] / buy_price < _lose_coef :
        sell_price = buy_price * _lose_coef
        sell_finished = True
        buy_order_placed = False
        number_buy_stocks = int(buy_money / buy_price)
        history_log['profit'] += (sell_price - buy_price) * number_buy_stocks - commission_value * (sell_price + buy_price) * number_buy_stocks
        history_log['number fails'] += 1
        history_log['transactions'].append({
          'type': 'fail',
          'buy_price' : round(buy_price, 2),
          'sell_price': round(sell_price, 2),
          'buy_time' : buy_time,
          'sell_time' : df['low'].index[i],
        })

    condition = False
    # dv = 0.04
    # dv = np.amin([amps7[ticker] - 1, amps30[ticker] - 1, amps100[ticker] - 1]) - 0.001
    # print(amps_7[ticker] - dist_from_max_7)
    try:
      
      speed30 = df['close'].iloc[i - 1] / df['close'].iloc[i - 30]
      speed100 = df['close'].iloc[i - 1] / df['close'].iloc[i - 100]
      ref30 = dist_from_max_30 / df_stats[ticker]['amps_avg30'].iloc[i - 1]

      current_amp30 = max30 / min30
      current_amp100 = max100 / min100
      norm30 = (current_amp30 - 1) / (df_stats[ticker]['amps_avg30'].iloc[i - 1] - 1)
      norm100 = (current_amp100 - 1) / (df_stats[ticker]['amps_avg100'].iloc[i - 1] - 1)
      avg_speed30 = df_stats[ticker]['speed30'].iloc[i - 1] 
      avg_speed100 = df_stats[ticker]['speed100'].iloc[i - 1] 
      # print(f'ratio 30 {norm30}')
      # print(f'ratio 100 {norm100}')
      # print(f'avg_speed30 is {avg_speed30}, avg_speed100 is {avg_speed100}')

      # if  speed30 < 1\
      #   and speed100 < 1\
      #   and norm30 > tp1 \
      #   and norm30 < tp2 \
      #   and norm100 > tp3 \
      #   and norm100 < tp4:

      # if  speed30 > 1\
      #   and speed100 < 1\
      #   and norm30 >= tp1 \
      #   and norm30 <= tp2 \
      #   and norm100 >= tp3 \
      #   and norm100 <= tp4 \
      #   and not (df.index[i - 1].hour in [15, 16]) \
      #   and max30 / cv <= tp5:
      
      # if  speed30 > 1\
      #   and speed100 < 1\
      #   and not (df.index[i - 1].hour in [15, 16]) \
      #   and max30 / cv <= 1.03 \
      #   and ( \
      #         # (norm100 >= tp3 and norm100 <= tp4) \
      #         (norm100 >= 0 and norm100 <= 0.3) \
      #       ):

      if extremum_minimum(df_stats[ticker]['speed30_avg30'], i - 1):
        # and df_stats[ticker]['speed30_avg30'].iloc[i - 1] > df_stats[ticker]['speed30_avg30'].iloc[i - 5]:

          # and df['close'].iloc[i - 1] / df['close'].iloc[i - 100] < 1 - (amps_7[ticker] - 1) \
        condition = True
    except Exception as e:
      print(e)
      condition = False

    if False:
      pass
      # conditions = []
      # c1 = False
      # if df.index[i].hour == 10 and df.index[i].minute == 30:
      #   CM930 = df['close'].iloc[i - 1]
      # #   c1 = df['close'].iloc[i - 1] / df['close'].iloc[i - 2] > 1.005 \
      # #       and df['close'].iloc[i - 1] / df['close'].iloc[i - 2] < max_930_value_to_buy
      # # conditions.append(c1)

      # c2 = False
      # if df.index[i].hour == 11 and df.index[i].minute == 30:
      #   # i : 11hours, i - 1 : 10hours, i - 2: 9hours, i - 3 : 15hours
      #   CM1030 = (df['close'].iloc[i - 1] + df['open'].iloc[i - 1]) / 2
      #   c2 = more_than_pct(CM1030, CM930, 0.21) \
      #     and  df['open'].iloc[i - 2] / df['close'].iloc[i - 2]  > 1.01 \
      #     and df['close'].iloc[i - 3] / df['close'].iloc[i - 2] < 1.002 \
      #     and df['close'].iloc[i - 3] / df['close'].iloc[i - 2] > 0.99
      # conditions.append(c2)
  
      # c31, c32 = False, False
      # if df.index[i].hour == 12 and df.index[i].minute == 30:
      #    # i : 12hours, i - 1 : 11hours, i - 2: 10hours, i - 3 : 9hours
      #   CM1130 = (df['close'].iloc[i - 1] + df['open'].iloc[i - 1]) / 2
      #   # c31 = more_than_pct(CM1130, CM1030, 0.01) and more_than_pct(CM1130, CM930, 0.45) \
      #   c31 = more_than_pct(CM1130, CM1030, tp1) and more_than_pct(CM1030, CM930, tp2) \
      #     #  and  df['open'].iloc[i - 3] / df['close'].iloc[i - 3]  > tp2 \
      #     #  and  df['open'].iloc[i - 3] / df['close'].iloc[i - 3]  < tp2 + tp4
      #   # {'tp2': np.float64(0.992), 'tp4': np.float64(0.998)}
      #   c32 = more_than_pct(CM1130, CM1030, 0.01) and more_than_pct(CM930, CM1030, 0.01) and more_than_pct(CM1130, CM930, tp) # local minumum (1.6)
      # conditions.append(c31)
      # conditions.append(c32)
    
      # c41, c42 = False, False
      # if df.index[i].hour == 13 and df.index[i].minute == 30:
      #   CM1230 = (df['close'].iloc[i - 1] + df['open'].iloc[i - 1]) / 2
      #   c41 = CM1230 > CM1130 and CM1230 > CM1030 and CM1230 > CM930
      #   # c42 = CM1230 > CM1130 and CM1230 < CM930

      #   c42 = more_than_pct(CM1230, CM1130, 0) and more_than_pct(CM930, CM1130, 0)  # local minumum 
      # conditions.append(c41)
      # conditions.append(c42)

      # c51, c52 = False, False
      # if df.index[i].hour == 14 and df.index[i].minute == 30:
      #   CM1330 = (df['close'].iloc[i - 1] + df['open'].iloc[i - 1]) / 2
      #   c51 = CM1330 > CM1230 and CM1330 > CM1130 and CM1330 > CM1030 and CM1330 > CM930
      #   c52 = CM1330 > CM1230 and CM1230 < CM930
      # conditions.append(c51)
      # conditions.append(c52)

      # condition = False
      # if df.index[i].hour == 13 and df.index[i - 1].hour == 12:
      # # moomoom time: i: 14:30; i-1: 13:30: i-2: 12:30; i-3: 11:30; i-4: 10:30 i-5 : 16:00 : i-6: 16:00 
      # # candle is green
      # # candle pct more than 0 
      # # more taht close 9:30
      # # sum pct 10:30 - 12:30  more than 0 
      # # 9:30 less than -0l77
      #   if df['close'].iloc[i - 1] > df['open'].iloc[i - 1] \
      #     and df['pct'].iloc[i - 1] > 0 \
      #     and df['pct'].iloc[i - 4 : i - 1].sum() > 0\
      #     and df['close'].iloc[i - 1] > df['close'].iloc[i - 5] \
      #     and (df['pct'].iloc[i - 4] < -0.5 \
      #         or df['close'].iloc[i - 6] / df['close'].iloc[i - 5] > 0.996)\
      #     and norm(df, 7) < 0.8 \
      #     and norm(df, 220) > tp or False:
      #     condition = True
      
      # previos candle is green
      # condition = False
      # if df['close'].iloc[i - 1] > df['open'].iloc[i - 1] \
      #   and df['pct'].iloc[i - 1] > 0 \
      #   and norm(df, 7) > tp \
      #   and norm(df, 220) <= 1:
      #   condition = True

      # condition2 = False
      # CM1 = (df['close'].iloc[i - 1] + df['open'].iloc[i - 1]) / 2
      # CM2 = (df['close'].iloc[i - 2] + df['open'].iloc[i - 2]) / 2
      # CM3 = (df['close'].iloc[i - 3] + df['open'].iloc[i - 3]) / 2
      # # if CM1 > CM2 and CM1 > CM3 and CM2 > CM3 \
      # if True \
      #   and df.index[i].hour != 15 \
      #   and df.index[i].hour != 9 \
      #   and df['close'].iloc[i - 4] < df['open'].iloc[i - 4]\
      #   and df['pct'].iloc[i - 1] > 0 \
      #   and df['pct'].iloc[i - 2] > 0 \
      #   and df['pct'].iloc[i - 3] > 0:
      #     condition2 = True

    if buy_order_placed == False \
      and condition:

      # buy_price = min(df['close'].iloc[i - 10: i].min(), df['open'].iloc[i - 10: i].min() )
      # buy_price = df['open'].iloc[i]
      buy_price = df['close'].iloc[i - 1]
      buy_time = df.index[i]
      
      # sell_finished = False
      # buy_i = i

      if df['low'].iloc[i] <= buy_price and df['high'].iloc[i] >= buy_price:
        buy_order_placed = True
        sell_finished = False
        buy_i = i
    
  history_log['proportion'] =   history_log['number wins'] / ( history_log['number fails'] + 1)

  # Simulation results 

  if print_sim_results:
    print('-'*50)
    print(f'Simulation results for ticker {ticker}:')
    # print(f'amp_7 is {amps7[ticker]}, amp_30 is {amps30[ticker]}, amp_100 is {amps100[ticker]},  ')
    print(history_log)

  if print_trans_history:
    for tran in history_log['transactions']:
      c.print(tran, color='blue')

  return history_log['profit'], history_log['number wins'], history_log['number fails']


In [23]:
buy_time_cond = True
buy_ratio_border = 7
distance_from_last_top  = 0  # NOT SIGNIFICANT?
last_top_ratio = 1.002 # last top / current open price  NOT SIGNIFICANT?
RIV = 0.1 # Reversal indication Value
is_near_global_max_prt = 96 # usd to be 96
bull_trend_coef = 1.12
last_top_24hours_proportion_border = 1

best_params = {'tp1': np.float64(-100), 'tp2': np.float64(100), 'tp3': np.float64(1.7), 'tp4': np.float64(2), 'tp5': 1.05, 'tp6': 1.07}
for param in best_params:
  vars()[param] = best_params[param]


parameters_list_search = {
  # 'tp1': np.arange(-0.5, 1.5, 0.1),
  # 'tp2': np.arange(1.5, -0.5, -0.1),
  # 'tp3': np.arange(-0.5, 1.5, 0.1),
  # 'tp4': np.arange(1.5, -0.5, -0.1),

  # 'tp1': np.arange(0, 0.1, 1),
  # 'tp2': np.arange(0.4, 0, -0.1),
  'tp2': [0]
  # 'tp3': np.arange(0, 2, 0.3),
  # 'tp4': np.arange(2, 0, -0.3),
  # 'tp5': np.arange(1.05, 0.99, -0.005)
  # 'tp6': np.arange(1.16, 1, -0.04),
  # 'tp3': np.arange(1.5, 2, 0.05),
  # 'tp4': np.arange(3, 1.5, -0.1),

}


In [29]:
stock_name_list = []
stock_name_list += ['GOOG','JPM','XOM','UNH','JNJ','V','AVGO','PG','LLY','MA','HD','CVX','MRK', 
                       'PEP','COST','ABBV','ADBE','KO','CRM','WMT','MCD','CSCO','BAC','PFE','TMO','ACN','NFLX','ABT','AMD','LIN','ORCL','CMCSA',
                       'TXN','DIS','WFC','DHR','PM','NEE','VZ','INTC','RTX','HON','LOW','UPS','INTU','SPGI','NKE','COP','QCOM','BMY','CAT','UNP','BA','ISRG',
                        'GE','IBM','AMGN','AMAT','MDT','SBUX','PLD','NOW','MS','DE','BLK','GS','T','LMT','AXP','SYK','ADI','TJX','ELV','MDLZ','GILD','ADP','MMC',
                        'C','AMT','CVS','VRTX','SCHW','LRCX','MO','TMUS','SLB', 'ETN', 'ZTS', 'CI', 'PYPL']

stock_name_list += ['FI','CB','SO','REGN','BSX','EQIX','BDX','PANW','DUK','EOG','MU','AON','ITW','CSX','SNPS','PGR','APD','KLAC','CME','NOC','CDNS','ICE','ATVI',
                       'CL','SHW','WM','HCA','TGT','FCX','FDX','F','MMM','CMG','EW','GM','MCK','NXPI','MCO','NSC','HUM','EMR','DXCM','PNC','PH','MPC','APH',
                       'ROP','FTNT','MCHP','PXD','USB','CCI','MAR','MSI','GD','PSA','JCI','PSX','SRE','ADSK','AZO','TDG','ECL','AJG','KMB','TEL','TT','AEP','EL','PCAR',
                       'OXY','TFC','CARR','D','IDXX','GIS','ON','COF','ADM','MNST','NUE','CTAS','AIG','EXC','VLO','MRNA','ANET','WMB','O','STZ','IQV','HLT','CHTR','WELL',
                       'BIIB','SPG','MSCI','DHI','ROK']
len(stock_name_list)
max_930_value_to_buy = 3

In [13]:
def calc_validation():
  #validation
  total_profit_valid = 0
  total_loses_valid = 0
  total_wins_valid = 0
  for ticker in stock_name_list_opt_valid:
    file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
    path = os.path.join(parent_path, folder_saved_models, file_name)
    df = load_data_one_stock(path)
  
    profit_valid, wins_valid, loses_valid = one_stock_simulation(df, ticker,
                                                    mode='valid',
                                                    _gain_coef=gain_coef,
                                                    _lose_coef=lose_coef)
    total_profit_valid += profit_valid
    total_loses_valid += loses_valid
    total_wins_valid += wins_valid
  total_proportion_valid = total_wins_valid / (total_loses_valid + 1)
  # print(f'total_profit_valid is {total_profit_valid}, total_proportion_valid is {total_proportion_valid}')
  return total_profit_valid, total_proportion_valid 

In [47]:
# BEST PARAMETERS OPTIMIZATION  and OPTIMAL STOCK LIST
gain_coef = 1.005
lose_coef = 0.995
best_profit = -10**5
best_proportion = 0
best_profit_valid = -10**5
best_proportion_valid = 0
result_improved = True
optimize_profit = True
optimize_proportion =  False
optimize_smart = False
ticker_stats = {}
stock_name_list_opt = stock_name_list[:]
prev_parameters = {}
parameters_list_best_history = []
# stock_name_list_opt = ['GOOG', 'XOM']
# stock_name_list_opt = ['AMD', 'AVGO']
# stock_name_list_opt = ['JPM']
# tp = 0.4
# stock_name_list_opt = ['AMD', 'AVGO', 'GOOG', 'XOM',
# 'BA', 'ON', 'MCHP', 'ADI', 'PANW', 'DHI', 'ANET', 'LRCX', 'LLY',
# 'MU', 'TXN', 'AIG', 'WMB', 'BSX', 'NKE', 'OXY', 'TT', 'AMAT', 'ETN', 'DE']
# stock_name_list_opt = ['BA', 'ON', 'MCHP', 'ADI', 'PANW', 'DHI', 'ANET', 'LRCX', 'LLY']

# 'MU', 'TXN', 'AIG', 'WMB', 'BSX', 'NKE', 'OXY', 'TT', 'AMAT', 'ETN', 'DE']
# stock_name_list_opt = ['GOOG']
# stock_name_list_opt = ['UNH']

# stock_name_list_opt = ['DHI', 'CDNS', 'ORCL', 'CMG', 'ADBE', 'APH', 'AMD', 'NOW', 'EMR', 'ETN', 'EW', 'AMAT', 'FTNT', 'AIG', 'AVGO', 'TT', 'DE', 'GM', 'CAT', 'ON', 'INTU', 'LRCX', 'PYPL', 'LLY', 'GOOG', 'VLO', 'C', 'BAC', 'PH']

# stock_name_list_opt = ['GOOG', 'AVGO', 'LLY', 'ADBE', 'CRM', 'BAC', 'NFLX', 'AMD']
# stock_name_list_opt = ['FI','CB','SO','REGN','BSX','EQIX','BDX','PANW','DUK','EOG','MU','AON','ITW','CSX','SNPS','PGR','APD','KLAC','CME','NOC','CDNS','ICE','ATVI',
#                        'CL','SHW','WM','HCA','TGT','FCX','FDX','F','ORLY','MMM','CMG','EW','GM','MCK','NXPI','MCO','NSC','HUM','EMR','DXCM','PNC','PH','MPC','APH',
#                        'ROP','FTNT','MCHP','PXD','USB','CCI','MAR','MSI','GD','PSA','JCI','PSX','SRE','ADSK','AZO','TDG','ECL','AJG','KMB','TEL','TT','AEP','EL','PCAR',
#                        'OXY','TFC','CARR','D','IDXX','GIS','ON','COF','ADM','MNST','NUE','CTAS','AIG','EXC','VLO','MRNA','ANET','WMB','O','STZ','IQV','HLT','CHTR','WELL',
#                        'BIIB','SPG','MSCI','DHI','ROK']
# profit > 0 
# stock_name_list_opt =['DHI', 'LRCX', 'CMG', 'GM', 'TGT', 'CDNS', 'CHTR', 'PANW', 'FTNT', 'HUM', 'PH', 'DXCM', 'FCX', 'AVGO', 'VLO', 'APH', 'BAC', 'IQV', 'NKE', 'PCAR', 'ROK', 'COF', 'C', 'GOOG', 'MS', 'CAT', 'NOW', 'GS', 'CARR', 'TFC', 'JCI', 'SLB', 'ON']
# profit < 0
# stock_name_list_opt = ['INTC', 'EMR', 'BA', 'USB', 'KLAC', 'EW', 'LLY', 'ETN', 'DE', 'MAR', 'TXN', 'PYPL', 'SBUX', 'AMD', 'EL', 'PNC', 'SCHW', 'ADI', 'NUE', 'WFC', 'ANET', 'QCOM', 'SNPS', 'F', 'AMAT', 'NXPI', 'BMY', 'MMM', 'MRNA', 'MCHP', 'MU']
# profit = 0 after first cycle
stock_name_list_opt = ['ABT', 'LIN', 'MDT', 'NFLX', 'XOM', 'UNH', 'PG', 'MA', 'JNJ', 'V', 'CRM', 'CVX', 'PEP', 'MRK', 'ABBV',
                        'ADBE', 'KO', 'COST', 'PFE', 'WMT', 'MCD', 'CSCO', 'ACN', 'TMO', 'HD', 'JPM', 'BIIB', 'WELL', 'CTAS',
                        'AIG', 'ADM', 'O', 'KMB', 'TT', 'ADSK', 'UNP', 'COP', 'ISRG', 'IBM', 'GE', 'AMGN', 'SPGI', 'DHR', 
                        'PM', 'INTU', 'UPS', 'VZ', 'LOW', 'HON', 'RTX', 'CMCSA', 'ORCL', 'NEE', 'DIS', 'MDLZ', 'ELV', 'T']

# stock_name_list_opt =['SYK', 'FDX', 'WM', 'PGR', 'APD', 'SHW', 'CME', 'REGN', 'SO', 'EOG', 'AON', 'ITW', 'CSX', 'BSX', 
#                           'EQIX', 'BDX', 'PLD', 'FI', 'MO', 'TMUS', 'MMC', 'AMT', 'CVS', 'VRTX', 'ADP', 'CI', 'ZTS', 'CB',
#                             'TJX', 'GILD', 'BLK', 'AXP', 'LMT', 'MSCI', 'SPG', 'STZ', 'HLT', 'WMB', 'EXC', 'D', 'AEP', 'MNST',
#                               'SRE', 'GIS', 'IDXX', 'ICE', 'NOC', 'HCA', 'CL', 'DUK', 'AZO', 'AJG', 'ECL', 'TEL', 'PSA', 'TDG',
#                                 'NSC', 'ROP', 'MPC', 'MCO', 'PSX', 'MSI', 'GD', 'CCI', 'MCK', 'OXY']

# stock_name_list_opt = ['GOOG', 'ABT', 'LIN', 'MDT', 'INTC', 'AVGO', 'AMD']
# stock_name_list_opt = ['AMD']

# for parameter in parameters_list_search:
counter = 0
while counter < 10:
  stock_name_list_opt = random.sample(stock_name_list, 1)
  # stock_name_list_opt = stock_name_list
  stock_name_list_opt_valid = random.sample(stock_name_list, 1)
  
  print(f'Current stock name list opt is {stock_name_list_opt}')
  print(f'Current stock name list validation is {stock_name_list_opt_valid}')

  best_profit = -10**5
  best_proportion = 0 
  for i in random.sample(range(len(parameters_list_search)), len(parameters_list_search)):
    parameter = list(parameters_list_search.keys())[i]
    total_profit_list = []
    total_loses_list = []
    total_proportion_list = []

    total_profit_valid_init, total_proportion_valid_init = calc_validation()
    print(f'Init valid params are, total_valid is {total_profit_valid_init:.3f},total_prop_valid is {total_proportion_valid_init:.3f}')
    # save prev params
    for par in parameters_list_search:
      prev_parameters[par] = vars()[par]

    cycle_number = -1
    for value in parameters_list_search[parameter]:
      cycle_number += 1 
      vars()[parameter] = value
      if parameter == 'tp1' and tp1 >= tp2: break
      if parameter == 'tp2' and tp2 <= tp1: break
      if parameter == 'tp3' and tp3 >= tp4: break
      if parameter == 'tp4' and tp4 <= tp3: break

      c.print(f'parametr is {parameter} value is {value:.3f}:', color='white')
      total_profit = 0
      total_loses = 0
      total_wins = 0

      for ticker in stock_name_list_opt:
        file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
        path = os.path.join(parent_path, folder_saved_models, file_name)
        df = load_data_one_stock(path)
        ticker_stats[ticker] = {
            'profit': -999,
            'wins': 0,
            'loses' : 0,
        }
        if df.shape[0] > 0:
          profit, wins, loses= one_stock_simulation(df, ticker,
                                                     mode='train',
                                                     _gain_coef=gain_coef,
                                                     _lose_coef=lose_coef)
          total_profit += profit
          total_loses += loses
          total_wins += wins
          ticker_stats[ticker] = {
            'profit': profit,
            'wins': wins,
            'loses' : loses,
          }
        else:
          try:
            stock_name_list.remove(ticker)
          except:
            pass
     
      total_profit_list.append(total_profit)
      total_loses_list.append(total_loses)
      total_proportion_list.append(total_wins / (total_loses + 1))
      c.print(f'Total Profit is {total_profit:.2f}, total wins is {total_wins}, total loses is {total_loses} \
      proportion {total_wins/(total_loses + 1):.3f}', color = 'red')
      if cycle_number == 0 and total_loses == 0 and total_wins == 0: break
    
    if total_profit_list:

      if optimize_smart:
        if max(total_profit_list) >= 0:
          optimize_proportion = False
          optimize_profit = True
        else:
          optimize_proportion = True
          optimize_profit = False

      if optimize_proportion:
        print('Proportion optimization')
        result_improved = max(total_proportion_list) >= best_proportion
        best_proportion = max(total_proportion_list) if result_improved else best_proportion
      else:
        print('Profit optimization')
        result_improved = max(total_profit_list) >= best_profit
        best_profit = max(total_profit_list) if result_improved else best_profit
    else:
      result_improved = False
      counter += 1
      #reverse parametes to save valeus
      c.print('parameters are reversed', color='red')
      for par in parameters_list_search:
        vars()[par] =  prev_parameters[par]
    
    if len(parameters_list_search) <=1:
      result_improved = False

    if result_improved:
      # using best params
      if optimize_profit:
        print('Profit optimization')
        vars()[parameter] = parameters_list_search[parameter][np.argmax(total_profit_list)]
      elif optimize_proportion:
        print('Proportion optimization')
        vars()[parameter] = parameters_list_search[parameter][np.argmax(total_proportion_list)]
      else:
        vars()[parameter] = parameters_list_search[parameter][np.argmin(total_loses_list)]
      print(f'parameter {parameter} best value is {vars()[parameter]:.3f}')

      total_profit_valid, total_proportion_valid = calc_validation()
      print(f'After optimization valid params are, total_profit_valid {total_profit_valid:.3f},total_prop_valid is {total_proportion_valid:.3f}')
      valid = False
      if optimize_profit:
        if total_profit_valid >= total_profit_valid_init or total_profit_valid_init == 0:
          valid = True
      if optimize_proportion or total_proportion_valid_init == 0 :
        if total_proportion_valid >= total_proportion_valid_init:
          valid = True

      if valid:
        parameters_list_best = {}
        for parameter in parameters_list_search:
          print(f'parameter {parameter} best value is {vars()[parameter]:.3f}')
          parameters_list_best[parameter] = vars()[parameter]
        parameters_list_best_history.append([parameters_list_best, total_profit_valid, total_proportion_valid])
        # print(parameters_list_best_history)
        print(parameters_list_best)
        print(max(total_profit_list), min(total_loses_list))
        prev_parameters = parameters_list_best
        with open(f'{folder_saved_models}/current_best_params.plk', 'wb') as file:
          pickle.dump(parameters_list_best, file) 
        counter = 0
      else:   
        counter += 1
        #reverse parametes to save valeus
        c.print('parameters are reversed', color='red')
        for par in parameters_list_search:
          vars()[par] =  prev_parameters[par]

  print(f'Counter is {counter}')
  # Optimization which stocks to use:
  # for ticker in stock_name_list_opt:
  #   if ticker_stats[ticker]['wins'] / (ticker_stats[ticker]['loses'] + 1) < 3 \
  #       or ticker_stats[ticker]['profit'] < 0:
  #     stock_name_list_opt.remove(ticker)
  # c.print(f'Current optimal stock list len is {len(stock_name_list_opt)}, list is {stock_name_list_opt}', color='green')

Current stock name list opt is ['NOW']
Current stock name list validation is ['VZ']
Init valid params are, total_valid is -190.088,total_prop_valid is 0.621
parametr is tp2 value is 0.000:
Total Profit is -236.57, total wins is 73, total loses is 71       proportion 1.014
Profit optimization
Counter is 0
Current stock name list opt is ['HCA']
Current stock name list validation is ['HUM']
Init valid params are, total_valid is -100.972,total_prop_valid is 0.929
parametr is tp2 value is 0.000:
Total Profit is -78.89, total wins is 79, total loses is 60       proportion 1.295
Profit optimization
Counter is 0
Current stock name list opt is ['GIS']
Current stock name list validation is ['SNPS']
Init valid params are, total_valid is -54.052,total_prop_valid is 1.148
parametr is tp2 value is 0.000:
Total Profit is -129.95, total wins is 51, total loses is 45       proportion 1.109
Profit optimization
Counter is 0
Current stock name list opt is ['KO']
Current stock name list validation is ['JPM

KeyboardInterrupt: 

In [83]:
for item in parameters_list_best_history:
  print(item)
# len(stock_name_list_opt)

[{'tp1': np.float64(1.7000000000000002), 'tp2': np.float64(2.0), 'tp3': np.float64(-1.0), 'tp4': np.float64(1.7)}, np.float64(76.21657587718785), 10.0]
[{'tp1': np.float64(1.7000000000000002), 'tp2': np.float64(2.0), 'tp3': np.float64(-1.0), 'tp4': np.float64(2.0)}, np.float64(91.58424486247777), 12.0]
[{'tp1': np.float64(1.7000000000000002), 'tp2': np.float64(2.0), 'tp3': np.float64(-1.0), 'tp4': np.float64(2.0)}, np.float64(91.58424486247777), 12.0]
[{'tp1': np.float64(1.7000000000000002), 'tp2': np.float64(2.0), 'tp3': np.float64(-1.0), 'tp4': np.float64(2.0)}, np.float64(91.58424486247777), 12.0]
[{'tp1': np.float64(1.7000000000000002), 'tp2': np.float64(2.0), 'tp3': np.float64(-1.0), 'tp4': np.float64(2.0)}, np.float64(120.0014177241864), 16.0]
[{'tp1': np.float64(1.7000000000000002), 'tp2': np.float64(2.0), 'tp3': np.float64(1.7000000000000002), 'tp4': np.float64(2.0)}, np.float64(120.0014177241864), 16.0]
[{'tp1': np.float64(1.7000000000000002), 'tp2': np.float64(2.0), 'tp3': np

In [107]:
for parameter in parameters_list_search:
  print(f'parameter {parameter} best value is {vars()[parameter]}')

  # if df['ha_pct'].iloc[i] > RIV \
  # and reverse_flag==False \
  # and sell_finished == True \
  # and last_top / df['open'].iloc[i] > last_top_ratio \
  # and i - last_top_i > distance_from_last_top \
  # and buy_time_cond \
  # and buy_ratio > buy_ratio_border \
  # and not(is_near_global_max(i, k=400, prt=is_near_global_max_prt)):

# Current_optimal_stock_list 
# stock_name_list_opt = ['JPM', 'UNH', 'AVGO', 'HD', 'PEP', 'COST', 'CRM', 'WMT',
#                                 'MCD', 'ABT', 'TXN', 'WFC', 'INTC', 'RTX', 'HON', 'SPGI',
#                                   'BA', 'AMGN', 'AMAT', 'SBUX', 'PLD', 'MS', 'DE', 'AXP', 
#                                   'ADI', 'TJX', 'MMC', 'CVS', 'SCHW', 'TMUS', 'ETN', 'CI', 
#                                   'BSX', 'AON', 'ITW', 'SNPS', 'KLAC', 'CDNS', 'WM', 'HCA',
#                                     'FDX', 'MMM', 'CMG', 'GM', 'MCK', 'NXPI', 'NSC',
#                                       'HUM', 'DXCM', 'PH', 'MPC', 'ROP', 'MCHP', 'MSI', 'GD',
#                                         'JCI', 'SRE', 'ADSK', 'AZO', 'TDG', 'AJG', 'TT', 'PCAR',
#                                           'COF', 'CTAS', 'AIG', 'ANET', 'SPG', 'MSCI', 'ROK']

stock_name_list_opt = [
'BA', 'ON', 'MCHP', 'ADI', 'PANW', 'DHI', 'ANET', 'AMD', 'LRCX', 'LLY',
'MU', 'TXN', 'AIG', 'WMB', 'BSX', 'NKE', 'OXY', 'TT', 'AMAT', 'ETN', 'DE',
'EL', 'FDX', 'MAR', 'GE', 'NFLX', 'NUE', 'GOOG', 'ECL', 'AVGO', 'CAT', 'SPG',
'ADSK', 'INTU', 'SLB', 'F', 'WMT', 'SBUX', 'SNPS', 'AJG', 'TMUS', 'KLAC', 'CI',
'JCI', 'GILD', 'QCOM', 'ROP', 'MO', 'WM', 'HON', 'ITW', 'GS', 'HCA', 'TJX', 'ICE',
'DXCM', 'IDXX', 'ABBV', 'CDNS', 'CMCSA', 'JNJ', 'EQIX', 'MDLZ', 'NXPI', 'MSI', 'TEL',
'LMT', 'USB', 'MRK', 'HLT', 'APD', 'CTAS', 'MNST', 'NOW', 'AMT', 'PH', 'HUM', 'ADM',
'TDG', 'EMR', 'GM', 'ADP', 'CMG', 'SCHW', 'MSCI', 'EOG', 'UNP', 'INTC', 'CME',
'MA', 'CVS', 'XOM', 'CSCO', 'WELL', 'TMO', 'MRNA', 'PLD', 'APH', 'PEP', 'CRM', 'MMM',
'MMC', 'LIN', 'GIS', 'COST', 'CSX', 'IQV', 'FI', 'MCD', 'VRTX'
]

# stock_name_list_opt =['GOOG', 'JPM', 'XOM', 'UNH', 'JNJ', 'V', 'AVGO', 'PG', 'LLY', 'MA', 'HD', 'CVX', 'MRK', 'PEP', 'COST', 'ABBV', 'ADBE', 'KO', 'CRM', 'WMT', 'MCD', 'CSCO', 'BAC', 'PFE', 'TMO', 'ACN', 'NFLX', 'ABT', 'AMD', 'LIN', 'ORCL', 'CMCSA', 'TXN', 'DIS', 'WFC', 'DHR', 'PM', 'NEE', 'VZ', 'INTC', 'RTX', 'HON', 'LOW', 'UPS', 'INTU', 'SPGI', 'NKE', 'COP', 'QCOM', 'BMY', 'CAT', 'UNP', 'BA', 'ISRG', 'GE', 'IBM', 'AMGN', 'AMAT', 'MDT', 'SBUX', 'PLD', 'NOW', 'MS', 'DE', 'BLK', 'GS', 'T', 'LMT', 'AXP', 'BKNG', 'SYK', 'ADI', 'TJX', 'ELV', 'MDLZ', 'GILD', 'ADP', 'MMC', 'C', 'AMT', 'CVS', 'VRTX', 'SCHW', 'LRCX', 'MO', 'TMUS', 'SLB', 'ETN', 'ZTS', 'CI', 'PYPL', 'FI', 'CB', 'SO', 'REGN', 'BSX', 'EQIX', 'BDX', 'PANW', 'DUK', 'EOG', 'MU', 'AON', 'ITW', 'CSX', 'SNPS', 'PGR', 'APD', 'KLAC', 'CME', 'NOC', 'CDNS', 'ICE', 'CL', 'SHW', 'WM', 'HCA', 'TGT', 'FCX', 'FDX', 'F', 'ORLY', 'MMM', 'CMG', 'EW', 'GM', 'MCK', 'NXPI', 'NSC', 'HUM', 'EMR', 'DXCM', 'PNC', 'PH', 'MPC', 'APH', 'ROP', 'FTNT', 'MCHP', 'USB', 'CCI', 'MAR', 'MSI', 'GD', 'PSA', 'JCI', 'PSX', 'SRE', 'ADSK', 'AZO', 'TDG', 'ECL', 'AJG', 'KMB', 'TEL', 'TT', 'AEP', 'EL', 'PCAR', 'OXY', 'TFC', 'CARR', 'D', 'IDXX', 'GIS', 'ON', 'COF', 'ADM', 'MNST', 'NUE', 'CTAS', 'AIG', 'EXC', 'VLO', 'MRNA', 'ANET', 'WMB', 'O', 'STZ', 'IQV', 'HLT', 'CHTR', 'WELL', 'BIIB', 'SPG', 'MSCI', 'DHI', 'ROK']


len(stock_name_list_opt), len(stock_name_list)


parameter tp3 best value is 1.4000000000000004
parameter tp4 best value is 1.7
parameter tp5 best value is 1.03


(110, 187)

In [47]:
tp1, tp2, tp3, tp4

(np.float64(1.7000000000000002),
 np.float64(2.0),
 np.float64(-1.0),
 np.float64(1.7))

In [101]:
len(stock_name_list), len(stock_name_list_opt)

(188, 110)

In [70]:
# TRY TO REDUCE NUMBER OF FAIL NOT PROFIT!!!
total_profit_list = []
# parametr_list = np.arange(86, 100, 1)
# parametr_list = [1, 1.001, 1.002, 1.005, 1.007, 1.01, 1.02]
# parametr_list = [30]
# for p in parametr_list:
#   is_near_global_max_prt= p
 
# print("TRAIN RESULTS")
# total_profit = 0
# total_loses = 0
# total_wins = 0
# for ticker in stock_name_list_opt:
#   file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
#   path = os.path.join(parent_path, folder_saved_models, file_name)
#   df = load_data_one_stock(path)
#   profit, wins, loses = one_stock_simulation(df, ticker, mode = 'train', print_sim_results=True, print_trans_history=False)
#   total_profit += profit
#   total_loses += loses
#   total_wins += wins
# print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}, total wins is {total_wins}, proportion: {total_wins / total_loses + 1:.2f}')

print("TEST RESULTS")
total_profit = 0
total_loses = 0
total_wins = 0
for ticker in stock_name_list_opt:
# for ticker in stock_name_list:
  file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
  path = os.path.join(parent_path, folder_saved_models, file_name)
  df = load_data_one_stock(path)
  if df.shape[0] > 0:
    profit, wins, loses = one_stock_simulation(df, ticker, mode = 'test', print_sim_results=True, print_trans_history=False)
    total_profit += profit
    total_loses += loses
    total_wins += wins
    print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}, total wins is {total_wins}, proportion: {total_wins / (total_loses + 1):.2f}')
  else:
    print(f'{ticker} empty df')


TEST RESULTS
--------------------------------------------------
Simulation results for ticker GOOG:
{'number success': 1, 'number fails': 1, 'number wins': 2, 'proportion': 1.0, 'profit': np.float64(-5.248949981689453), 'transactions': [{'type': 'success', 'buy_price': np.float64(129.08), 'sell_price': np.float64(129.46), 'buy_time': Timestamp('2023-09-26 12:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2023-09-26 15:30:00-0400', tz='America/New_York')}, {'type': 'success', 'buy_price': np.float64(133.83), 'sell_price': np.float64(133.32), 'buy_time': Timestamp('2023-10-03 12:30:00-0400', tz='America/New_York'), 'sell_time': Timestamp('2023-10-03 15:30:00-0400', tz='America/New_York')}]}
Total Profit is -5.25, total loses is 1, total wins is 2, proportion: 1.00
--------------------------------------------------
Simulation results for ticker JPM:
{'number success': 0, 'number fails': 1, 'number wins': 1, 'proportion': 0.5, 'profit': np.float64(-6.696629859924316), 'transa

In [31]:
# best_params = {'is_near_global_max_prt': 96, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.05, 'buy_ratio_border': 0, 'bull_trend_coef': 1.07}
# best_params = {'is_near_global_max_prt': 30, 'distance_from_last_top': 0, 'last_top_ratio': 1.1, 'RIV': 0.1, 'buy_ratio_border': 0, 'bull_trend_coef': 1.07}
# best_params = {'is_near_global_max_prt': 20, 'distance_from_last_top': 0, 'last_top_ratio': 1.1, 'RIV': 0.05, 'buy_ratio_border': 0, 'bull_trend_coef': 1.07}
# best_params = {'is_near_global_max_prt': 70, 'distance_from_last_top': 0, 'last_top_ratio': 1.1, 'RIV': 0.2, 'buy_ratio_border': 7, 'bull_trend_coef': 1.07}
best_params = {'is_near_global_max_prt': 120, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV': 0.9, 'buy_ratio_border': 0, 'bull_trend_coef': 1.07,
              'is_near_global_max_prt2': 60,  'global_max_prop_ratio' : 1.08, 'nrc': 6}
for param in best_params:
  vars()[param] = best_params[param]

stock_name_list_opt = ['MAR', 'MO', 'DHI', 'PYPL', 'HLT', 'BIIB', 'ZTS', 'TMUS', 'CMG', 'WELL', 'DUK',
                       'MCHP', 'MS', 'ROK', 'INTU', 'EXC', 'CI', 'STZ', 'NXPI', 'COF', 'DIS', 'EL', 'MU',
                       'NOW', 'CAT', 'HUM', 'OXY', 'PSX', 'COP', 'QCOM', 'ON', 'CVS', 'VLO', 'TJX', 'PGR',
                       'TXN', 'EOG', 'CSX', 'PM', 'MMM', 'ADM', 'GM', 'PEP', 'VZ', 'REGN', 'PCAR', 'GILD',
                       'EW', 'ANET', 'GIS', 'LIN', 'TEL', 'VRTX', 'BLK', 'ELV', 'KO', 'DE', 'KLAC', 'SBUX',
                       'PLD', 'PANW', 'CHTR', 'SNPS', 'MCK', 'TFC', 'USB', 'TDG', 'PFE', 'MRK', 'CB', 'MPC',
                       'GOOG', 'TT', 'PSA', 'CME', 'AIG', 'IDXX', 'BSX', 'WFC', 'ABT', 'TGT', 'EQIX', 'CTAS',
                       'BA', 'AON', 'SYK', 'F', 'SRE', 'JCI', 'NKE', 'MCD', 'TMO', 'SO', 'WMT', 'UNH', 'CSCO',
                      'CRM', 'PG', 'JNJ', 'HCA', 'XOM', 'ETN', 'V', 'COST', 'GS', 'FCX', 'MDT', 'NEE', 'CL',
                       'BMY', 'SPG', 'ADI', 'ABBV', 'WM', 'NUE', 'MDLZ', 'MA', 'MRNA', 'ROP', 'AEP', 'IBM',
                      'EMR', 'JPM', 'APD', 'AMD', 'T', 'INTC', 'ITW', 'AMGN', 'ADSK', 'SHW', 'NOC', 'NSC', 'SCHW', 'ICE']

In [48]:
# best_params = {'is_near_global_max_prt': 120, 'distance_from_last_top': 0, 'last_top_ratio': 1, 'RIV1': 0.3, 'RIV2': 0.9, 'buy_ratio_border': 0, 'bull_trend_coef': 1.07,
#               'is_near_global_max_prt2': 60,  'global_max_prop_ratio' : 1.08, 'nrc': 3, 'tp1': 0, 'tp2': 0, 'tp': 1}
# for param in best_params:
#   vars()[param] = best_params[param]

# best_params = {'tp1': 0.005, 'tp2': 0.04, 'tp3': 0.94, 'tp4': 1.006, 'tp5' : 1.0015, 'tp6' : 1.006}
# best_params = {'tp1': np.float64(0.0), 'tp2': np.float64(0.045000000000000005), 'tp4': np.float64(1.008999999999999), 'tp5': 1.0015, 'tp6': np.float64(1.0029999999999997)}
# best_params ={'tp2': np.float64(0.08500000000000002), 'tp4': np.float64(1.002), 'tp5': 1.0015, 'tp6': np.float64(1.003)}
# best_params = {'tp1': np.float64(0.009), 'tp2': np.float64(0.085), 'tp3': np.float64(0.95), 
#                'tp4': np.float64(1.002), 'tp5': 1.0015, 'tp6': np.float64(1.003)}
# best_params = {'tp2': np.float64(0.075), 'tp4': np.float64(1.004), 'tp5': 1.0015, 'tp6': np.float64(1.005)}
best_params = {'tp1': np.float64(0.7999999999999998), 'tp2': np.float64(1.1999999999999997), 'tp3': np.float64(-0.5), 'tp4': np.float64(0.5999999999999992)}
best_params = {'tp1': np.float64(0.4999999999999998), 'tp2': np.float64(0.6999999999999993), 'tp3': np.float64(0.3999999999999998), 'tp4': np.float64(0.4999999999999991)}
best_params = {'tp1': np.float64(0.8999999999999997), 'tp2': np.float64(1.4), 'tp3': np.float64(0.5999999999999996), 'tp4': np.float64(1.5)}
best_params = {'tp1': np.float64(1.1999999999999997), 'tp2': np.float64(1.4), 'tp3': np.float64(-0.5), 'tp4': np.float64(1.4)}
best_params = {'tp1': np.float64(0.7999999999999998), 'tp2': np.float64(0.9999999999999996), 'tp3': np.float64(0.4999999999999998), 'tp4': np.float64(0.9999999999999996)}
best_params = {'tp1': np.float64(-0.5), 'tp2': np.float64(-0.6000000000000017), 'tp3': np.float64(-0.5), 'tp4': np.float64(0.6)}
best_params = {'tp1': np.float64(-0.5), 'tp2': np.float64(0.5), 'tp3': np.float64(-0.5), 'tp4': np.float64(1.5)} # 16.74 proportion
best_params = {'tp1': np.float64(-0.5), 'tp2': np.float64(0.6999999999999993), 'tp3': np.float64(-0.5), 'tp4': np.float64(1.5)} # 7.46 proportion
best_params = {'tp1': np.float64(-0.5), 'tp2': np.float64(1.5), 'tp3': np.float64(0.7), 'tp4': np.float64(0.8)} #  proportion
best_params = {'tp1': np.float64(-1.0), 'tp2': np.float64(2.0), 'tp3': np.float64(1.5), 'tp4': np.float64(2.0)} # 7.16 full, 8.69 test proportion
best_params = {'tp1': np.float64(1.1999999999999993), 'tp2': np.float64(1.4999999999999996), 'tp3': np.float64(-1.0), 'tp4': np.float64(2.0)} #  full,  7.10 test proportion
best_params = {'tp1': np.float64(0.49999999999999956), 'tp2': np.float64(0.5999999999999988), 'tp3': np.float64(-1.0), 'tp4': np.float64(0.49999999999999867)} #  full,  test proportion
best_params = {'tp1': np.float64(-1.0), 'tp2': np.float64(0.4999999999999998), 'tp3': np.float64(1.4000000000000004), 'tp4': np.float64(2.0)} #  full,  test proportion
  #  if  speed30 > 1\
  #  speed100 < 1\
best_params = {'tp1': np.float64(-1.0), 'tp2': np.float64(2.0), 'tp3': np.float64(1.7000000000000002), 'tp4': np.float64(2.0)} #  full, 7.71 proportion: 11.86 test proportion
  #  if  speed30 > 1\
  #  speed100 > 1\
best_params = {'tp1': np.float64(-1.0), 'tp2': np.float64(0.4999999999999998), 'tp3': np.float64(-1.0), 'tp4': np.float64(2.0)}#  full, 6.46 proportion:  6.57 test proportion
  #  if  speed30 < 1\
  #  speed100 > 1\
best_params = {'tp1': np.float64(0.5000000000000002), 'tp2': np.float64(0.7999999999999998), 'tp3': np.float64(1.4000000000000004), 'tp4': np.float64(2.0)}#  full, 8.66 proportion:   8.09 test proportion
best_params = {'tp1': np.float64(-1.0), 'tp2': np.float64(0.4999999999999998), 'tp3': np.float64(-1.0), 'tp4': np.float64(0.7999999999999998)} # full, proportion:  6.48   test proportion 6.61
  #  if  speed30 < 1\
  #  speed100 < 1\
best_params = {'tp1': np.float64(-1.0), 'tp2': np.float64(2.0), 'tp3': np.float64(1.7000000000000002), 'tp4': np.float64(2.0)} # Total Profit is 375.33, total loses is 136, total wins is 1111, proportion: 8.17  test proportion 7.75
best_params = {'tp1': np.float64(-1.0), 'tp2': np.float64(0.4999999999999998), 'tp3': np.float64(-1.0), 'tp4': np.float64(2.0)} # test 7.94
best_params = {'tp1': np.float64(-1.0), 'tp2': np.float64(0.40000000000000036), 'tp3': np.float64(-1.0), 'tp4': np.float64(0.6000000000000003)} #  test 6.95 full 6.06
best_params = {'tp1': np.float64(-1.0), 'tp2': np.float64(0.6000000000000003), 'tp3': np.float64(-1.0), 'tp4': np.float64(2.0)} # test 8.27 # 6.86 full
best_params = {'tp1': np.float64(-1.0), 'tp2': np.float64(2), 'tp3': np.float64(1.2), 'tp4': np.float64(2.0)} # test 8.27 # 6.86 full         
# 3% Total Profit is 6765.85, total loses is 351, total wins is 501, proportion: 1.43
# Total Profit is 591.12, total loses is 271, total wins is 2199, proportion: 8.11
# best_params = {'tp1': np.float64(1.7000000000000002), 'tp2': np.float64(2.0), 'tp3': np.float64(-1.0), 'tp4': np.float64(2.0)}
  #  if  speed30 > 1\
  #  speed100 < 1\
best_params = {'tp1': np.float64(-1.0), 'tp2': np.float64(2.0), 'tp3': np.float64(1.7000000000000002), 'tp4': np.float64(2.0)} #  full, 7.71 proportion: 11.86 test proportion
# test : Total Profit is 686.74, total loses is 36, total wins is 366, proportion: 10.17
best_params = {'tp1': np.float64(-10), 'tp2': np.float64(10), 'tp3': np.float64(1.7000000000000002), 'tp4': np.float64(2.0)} #  full, 7.71 proportion: 11.86 test proportion
# Total Profit is 768.88, total loses is 36, total wins is 377, proportion: 10.47\
best_params = {'tp1': np.float64(-100), 'tp2': np.float64(100), 'tp3': np.float64(0), 'tp4': np.float64(0.5), 'tp5': 1.03} #  full, proportion:  test proportion
# TEST Total Profit is 883.04, total loses is 31, total wins is 354, proportion: 11.42


best_params = {'tp1': np.float64(-100), 'tp2': np.float64(100), 'tp3': np.float64(1.7), 'tp4': np.float64(2.0), 'tp5': 1.03}
# TEST Total Profit is 980.43, total loses is 31, total wins is 367, proportion: 11.84
# FULL Total Profit is -152.90, total loses is 101, total wins is 762, proportion: 7.54

best_params = {'tp1': np.float64(-100), 'tp2': np.float64(100), 'tp3': np.float64(1.7), 'tp4': np.float64(2.0), 'tp5': 1.05}
# TEST Total Profit is 810.99, total loses is 35, total wins is 374, proportion: 10.69
# FUll Total Profit is -319.68, total loses is 109, total wins is 800, proportion: 7.34

best_params = {'tp1': np.float64(-100), 'tp2': np.float64(100), 'tp3': np.float64(0), 'tp4': np.float64(0.5), 'tp5': 1.03} #  full, proportion:  test proportion
# TEST Total Profit is -541.93, total loses is 76, total wins is 523, proportion: 6.88
# FULL Total Profit is -2704.86, total loses is 210, total wins is 1274, proportion: 6.07

best_params = {'tp1': np.float64(-100), 'tp2': np.float64(100), 'tp3': np.float64(1.7), 'tp4': np.float64(4.0), 'tp5': 1.03}
# TEST Total Profit is 954.67, total loses is 58, total wins is 571, proportion: 9.84
# FULL Total Profit is -1375.65, total loses is 173, total wins is 1161, proportion: 6.71

# Static model tp5 = 1.03, no 15,16 hours.
# test: Total Profit is 430.56, total loses is 107, total wins is 889, proportion: 8.31
# full: Total Profit is -2835.80, total loses is 310, total wins is 2031, proportion: 6.55

# Static model tp5 = 1.005, no 15,16 hours.
# test: Total Profit is -108.03, total loses is 59, total wins is 449, proportion: 7.61

# Static model tp5 = 1.03, with 15,16 hours.
# Test Total Profit is 397.66, total loses is 115, total wins is 948, proportion: 8.24

# stocks selects
# best_params = {'tp1': np.float64(-100), 'tp2': np.float64(100), 'tp3': np.float64(1.6), 'tp4': np.float64(2), 'tp5': 1.03}
# TEST Total Profit is 909.91, total loses is 50, total wins is 503, proportion: 10.06
# best_params = {'tp1': np.float64(-100), 'tp2': np.float64(100), 'tp3': np.float64(1.7), 'tp4': np.float64(4), 'tp5': 1.03}
# TEST Total Profit is 954.67, total loses is 58, total wins is 571, proportion: 9.84 worse
for param in best_params:
  vars()[param] = best_params[param]

In [28]:
print("FULL RESULTS OPTIMIZATION")
# parameters:
mode = 'full'
lose_coef = 0.97
gain_coef = 1.005
# gain_coef = 1.02
use_optimal_stock_list = False
use_stock_name_list_selected = False
total_profit = 0
total_loses = 0
total_wins = 0

# tp1 = np.float64(0.21)
# tp2 =  np.float64(1.01)
tp = 0.3

if use_optimal_stock_list:
  current_stock_name_list = stock_name_list_opt[:]
else:
  current_stock_name_list = stock_name_list[:]

if use_stock_name_list_selected:
  current_stock_name_list = stock_name_list_selected[:]

# current_stock_name_list = ['CME', 'AMT']
# current_stock_name_list = ['AMD', 'JPM']
# current_stock_name_list =  ['ABT', 'LIN', 'MDT', 'INTC', 'AVGO', 'AMD']
# current_stock_name_list = ['GOOG', 'AVGO', 'LLY', 'ADBE', 'CRM', 'BAC', 'NFLX', 'AMD', 'ORCL', 'TXN', 'WFC', 'INTC', 'INTU', 'QCOM', 'BMY', 'CAT', 'BA', 'ISRG', 'GE', 'AMAT', 'SBUX', 'PLD', 'NOW', 'MS', 'DE', 'GS', 'AXP', 'ADI', 'C', 'CVS', 'SCHW', 'LRCX', 'SLB', 'ETN', 'PYPL', 'PANW', 'MU', 'SNPS', 'APD', 'KLAC', 'CDNS', 'TGT', 'FCX', 'F', 'MMM', 'CMG', 'EW', 'GM', 'MCK', 'NXPI', 'NSC', 'HUM', 'EMR', 'DXCM', 'PH', 'MPC', 'APH', 'FTNT', 'MCHP', 'USB', 'JCI', 'PSX', 'ADSK', 'TDG', 'TT', 'EL', 'PCAR', 'OXY', 'TFC', 'CARR', 'IDXX', 'ON', 'COF', 'MNST', 'NUE', 'AIG', 'VLO', 'MRNA', 'ANET', 'IQV', 'CHTR', 'BIIB', 'DHI', 'ROK']
# current_stock_name_list = ['GOOG', 'AVGO', 'LLY', 'ADBE', 'CRM', 'BAC', 'NFLX', 'AMD']
# current_stock_name_list = ['DHI', 'CDNS', 'ORCL', 'CMG', 'ADBE', 'APH', 'AMD', 'NOW', 'EMR', 'ETN', 'EW', 'AMAT', 'FTNT', 'AIG', 'AVGO', 'TT', 'DE', 'GM', 'CAT', 'ON', 'INTU', 'LRCX', 'PYPL', 'LLY', 'GOOG', 'VLO', 'C', 'BAC', 'PH']
# current_stock_name_list  = ['ABT', 'LIN', 'MDT']
# current_stock_name_list  = ['ABT', 'LIN', 'MDT', 'NFLX', 'XOM', 'UNH', 'PG', 'MA', 'JNJ', 'V', 'CRM', 'CVX', 'PEP', 'MRK', 'ABBV',
#                         'ADBE', 'KO', 'COST', 'PFE', 'WMT', 'MCD', 'CSCO', 'ACN', 'TMO', 'HD', 'JPM', 'BIIB', 'WELL', 'CTAS',
#                         'AIG', 'ADM', 'O', 'KMB', 'TT', 'ADSK', 'UNP', 'COP', 'ISRG', 'IBM', 'GE', 'AMGN', 'SPGI', 'DHR', 
#                         'PM', 'INTU', 'UPS', 'VZ', 'LOW', 'HON', 'RTX', 'CMCSA', 'ORCL', 'NEE', 'DIS', 'MDLZ', 'ELV', 'T']
# current_stock_name_list  =['SYK', 'FDX', 'WM', 'PGR', 'APD', 'SHW', 'CME', 'REGN', 'SO', 'EOG', 'AON', 'ITW', 'CSX', 'BSX', 
#                           'EQIX', 'BDX', 'PLD', 'FI', 'MO', 'TMUS', 'MMC', 'AMT', 'CVS', 'VRTX', 'ADP', 'CI', 'ZTS', 'CB',
#                             'TJX', 'GILD', 'BLK', 'AXP', 'LMT', 'MSCI', 'SPG', 'STZ', 'HLT', 'WMB', 'EXC', 'D', 'AEP', 'MNST',
#                               'SRE', 'GIS', 'IDXX', 'ICE', 'NOC', 'HCA', 'CL', 'DUK', 'AZO', 'AJG', 'ECL', 'TEL', 'PSA', 'TDG',
#                                 'NSC', 'ROP', 'MPC', 'MCO', 'PSX', 'MSI', 'GD', 'CCI', 'MCK', 'OXY']

# current_stock_name_list  = ['ABT', 'LIN', 'MDT', 'NFLX', 'XOM', 'UNH', 'PG', 'MA', 'JNJ', 'V', 'CRM', 'CVX', 'PEP', 'MRK', 'ABBV',
#                         'ADBE', 'KO', 'COST', 'PFE', 'WMT', 'MCD', 'CSCO', 'ACN', 'TMO', 'HD', 'JPM', 'BIIB', 'WELL', 'CTAS',
#                         'AIG', 'ADM', 'O', 'KMB', 'TT', 'ADSK', 'UNP', 'COP', 'ISRG', 'IBM', 'GE', 'AMGN', 'SPGI', 'DHR', 
#                         'PM', 'INTU', 'UPS', 'VZ', 'LOW', 'HON', 'RTX', 'CMCSA', 'ORCL', 'NEE', 'DIS', 'MDLZ', 'ELV', 'T']

# current_stock_name_list  =['DHI', 'FCX', 'SYK']
df_opt = pd.DataFrame(columns=['ticker','profit', 'wins', 'loses', 'proportion'])

for ticker in current_stock_name_list:
  file_name = f'df_stock_{ticker}_period2y_interval1h.pkl'
  path = os.path.join(parent_path, folder_saved_models, file_name)
  df = load_data_one_stock(path)

  # if ticker in opt_stocks_for_bear_trend:
  #   gain_coef = 1.02
  # else:
  #   gain_coef = 1.005

  if df.shape[0] > 0:
    try:
      profit, wins, loses = one_stock_simulation(df, ticker,
                                                mode=mode,
                                                _gain_coef=gain_coef,
                                                _lose_coef=lose_coef, 
                                                print_sim_results=True, 
                                                print_trans_history=True)
    except:
      profit, wins, loses = 0 ,0 ,0
    if loses == 0:
      proportion = wins
    else:
      proportion = wins / loses
    df2 = pd.DataFrame({'ticker': [ticker],'profit': [profit], 'wins': [wins], 'loses': [loses], 'proportion': [proportion]})
    df_opt = pd.concat([df_opt, df2], ignore_index=True)
    total_profit += profit
    total_loses += loses
    total_wins += wins
    if total_loses == 0:
      total_proportion = total_wins
    else:
      total_proportion = total_wins / total_loses
  else:
    print(f'{ticker} empty df')
c.print(f'Total Profit is {total_profit:.2f}, total loses is {total_loses}, total wins is {total_wins}, proportion: {total_proportion:.2f}', color = 'blue')
c.print('Parameters are:', color = 'blue' )
c.print(f'gain_coef is {gain_coef}; lose_coef is {lose_coef}; use_optimal_stock_list is {use_optimal_stock_list}, use_selected_optimal_stock_list is \
{use_stock_name_list_selected} mode is {mode}', color = 'blue')
for param in best_params:
   print(f'{param} is {vars()[param]}')

FULL RESULTS OPTIMIZATION
--------------------------------------------------
Simulation results for ticker GOOG:
{'number success': 0, 'number fails': 0, 'number wins': 0, 'proportion': 0.0, 'profit': 0, 'transactions': []}
--------------------------------------------------
Simulation results for ticker JPM:
{'number success': 0, 'number fails': 0, 'number wins': 0, 'proportion': 0.0, 'profit': 0, 'transactions': []}
--------------------------------------------------
Simulation results for ticker XOM:
{'number success': 0, 'number fails': 0, 'number wins': 0, 'proportion': 0.0, 'profit': 0, 'transactions': []}
--------------------------------------------------
Simulation results for ticker UNH:
{'number success': 0, 'number fails': 1, 'number wins': 0, 'proportion': 0.0, 'profit': np.float64(-47.21928192123413), 'transactions': [{'type': 'fail', 'buy_price': np.float64(507.98), 'sell_price': np.float64(492.74), 'buy_time': Timestamp('2023-08-16 10:30:00-0400', tz='America/New_York'), '

Results:
1. use ordered selected stock list;
2. Use danamic gain coeficient'


In [189]:
df_opt = df_opt.sort_values(by='profit', ascending=False)
print(df_opt)
df_opt.to_csv('df_opt.csv')

    ticker      profit wins loses proportion
185    DHI  122.691973   37    14   2.642857
82    LRCX  103.170779   32    12   2.666667
121    CMG   87.467033   14     2        7.0
123     GM   66.900268   40    21   1.904762
116    TGT   62.808471   28    13   2.153846
..     ...         ...  ...   ...        ...
49     BMY  -74.872181   16    17   0.941176
120    MMM  -79.410499   11    14   0.785714
173   MRNA  -82.957845   68    52   1.307692
137   MCHP  -98.021634   34    31   1.096774
100     MU -103.039634   39    35   1.114286

[187 rows x 5 columns]


In [31]:
stock_name_list_selected = df_opt.loc[df_opt['profit'] > 0].sort_values(by='profit', ascending=False)['ticker']
print(stock_name_list_selected.to_list())
print(len(stock_name_list_selected))
# stock_name_for_check = ['EXC', 'MCHP', 'ADI', 'WFC', 'EW', 'MNST', 'FTNT', 'AIG', 'BAC', 'TXN', 
#                         'TT', 'ECL', 'GOOG', 'LRCX', 'GS', 'PSX', 'MA', 'CVS', 'PG', 'BA', 'DE',
#                          'PGR', 'PFE', 'TFC', 'ADM', 'NOW', 'SRE', 'PLD', 'TEL', 'MCD', 'ADP', 'NOC', 
#                          'F', 'REGN', 'KO', 'ZTS', 'WM', 'CVX', 'NFLX', 'CCI', 'MDLZ', 'WMB', 'BSX', 'CARR',
#                          'PM', 'JNJ', 'MRK', 'COP', 'AON', 'COF', 'SYK', 'IQV', 'GD', 'V', 'VRTX', 'CSCO',
#                          'USB', 'TGT', 'DUK', 'MDT', 'ABT', 'PSA', 'KMB', 'MCK', 'KLAC', 'C', 'AEP', 'CHTR',
#                          'EOG', 'AJG', 'MU', 'ADBE', 'QCOM', 'ADSK', 'PCAR', 'GM', 'SBUX', 'AXP', 'CTAS',
#                          'FI', 'SPG', 'ORCL', 'NSC', 'IBM', 'INTU', 'BDX']
# (stock_name_for_check == stock_name_list_selected).all()

# print(stock_name_list_selected)
# df_opt.loc[df_opt['profit'] > 0].sort_values(by='profit', ascending=False)


['EXC', 'MCHP', 'ADI', 'WFC', 'EW', 'MNST', 'FTNT', 'AIG', 'BAC', 'TXN', 'TT', 'ECL', 'GOOG', 'LRCX', 'GS', 'PSX', 'MA', 'CVS', 'PG', 'BA', 'DE', 'PGR', 'PFE', 'TFC', 'ADM', 'NOW', 'SRE', 'PLD', 'TEL', 'MCD', 'ADP', 'NOC', 'F', 'REGN', 'KO', 'ZTS', 'WM', 'CVX', 'NFLX', 'CCI', 'MDLZ', 'WMB', 'BSX', 'CARR', 'PM', 'JNJ', 'MRK', 'COP', 'AON', 'COF', 'SYK', 'IQV', 'GD', 'V', 'VRTX', 'CSCO', 'USB', 'TGT', 'DUK', 'MDT', 'ABT', 'PSA', 'KMB', 'MCK', 'KLAC', 'C', 'AEP', 'CHTR', 'EOG', 'AJG', 'MU', 'ADBE', 'QCOM', 'ADSK', 'PCAR', 'GM', 'SBUX', 'AXP', 'CTAS', 'FI', 'SPG', 'ORCL', 'NSC', 'IBM', 'INTU', 'BDX']
86


np.True_

In [120]:
df_opt_bear_trend = df_opt.sort_values(by='proportion', ascending=False)

df_opt_bear_trend = df_opt_bear_trend.loc[df_opt_bear_trend['proportion'] >= 3].sort_values(by='profit', ascending=False)
print(df_opt_bear_trend)
print(df_opt_bear_trend['ticker'].to_list())

    ticker      profit wins loses  proportion
0       BA  674.602803   38     6    6.333333
8     INTU  657.527402   40     7    5.714286
6     MCHP  599.032862   41     8    5.125000
13     LLY  587.327168   26     4    6.500000
2      DHI  573.843221   36     8    4.500000
17    ANET  563.029703   36    12    3.000000
11     AIG  491.698669   32     7    4.571429
24     NUE  465.124099   39    12    3.250000
31     MAR  456.696729   33     6    5.500000
1      OXY  455.211293   36    10    3.600000
3       ON  444.715145   45    13    3.461538
21      GE  440.864404   28     5    5.600000
15    AMAT  410.183924   36    11    3.272727
10    NXPI  404.862433   38    10    3.800000
28    SNPS  376.630274   33    10    3.300000
87     UNP  360.880210   23     6    3.833333
30    KLAC  358.535270   32     8    4.000000
7      BSX  349.135627   20     4    5.000000
61     MSI  348.189323   26     5    5.200000
85     CRM  330.056228   33    11    3.000000
43     CAT  329.017333   26     7 